In [2]:
# Bibliotecas para baixar o dataset, graficos e leitura do dataset

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [3]:
# Baixa os dados de treinamento
file_path_train = 'C:/Treinamento-Data-Science/Atividade 4/aug_train.csv'
df_train = pd.read_csv(file_path_train)
df_train.shape

(382154, 12)

In [4]:
# Baixa os dados de teste
file_path_test = 'C:/Treinamento-Data-Science/Atividade 4/aug_test.csv'
df_test = pd.read_csv(file_path_test)
df_test.shape

(78273, 11)

In [5]:
# Três primeiras linhas do dataset de Treino
df_train.sample(3)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
143810,317575,Female,20,1,8.0,1,< 1 Year,No,32153.0,160.0,41,0
187818,85088,Female,27,1,26.0,1,< 1 Year,No,39279.0,152.0,298,0
58444,169093,Female,64,1,28.0,0,> 2 Years,Yes,336395.0,122.0,220,0


In [6]:
# Três primeiras linhas do dataset de Teste
df_test.sample(3)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
69230,150442,Male,45,1,46.0,0,1-2 Year,Yes,38740.0,124.0,79
55436,32417,Male,23,1,28.0,1,< 1 Year,Yes,23982.0,152.0,198
1737,467047,Male,33,1,29.0,1,< 1 Year,No,2630.0,152.0,54


In [7]:
# Merge dos Datasets
df_test['Response'] = 2 # Criando coluna "Response" no dataset de teste para concatenar com treino
frames = [df_train, df_test]
df_full = pd.concat(frames)
df_full.sample(3)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
49772,291662,Female,40,1,28.0,0,1-2 Year,Yes,28344.0,124.0,159,2
24509,168770,Female,28,1,43.0,1,< 1 Year,No,2630.0,152.0,197,0
297646,239735,Male,34,1,46.0,0,1-2 Year,Yes,2630.0,124.0,284,1


In [8]:
# Contagem dos valores
df_full['Response'].value_counts()

Response
0    319553
2     78273
1     62601
Name: count, dtype: int64

In [9]:
# Antes de Executar a Instalação, executar o DataSpell como ADM
# Executar a célula com o PIP Install
# Reiniciar o ambiente de execução (Se necessário)
# Comentar essa célula e executar tudo novamente
# !pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [10]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df_full, title="Pandas Profiling Report", explorative=True)

C:\Users\mathe\AppData\Local\Temp\ipykernel_14992\2506274947.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [11]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Program Files\Python311\Lib\site-packages\ydata_profiling\model\correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'cannot reindex on an axis with duplicate labels')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

#### Feature engineering

In [12]:
import pandas as pd
import seaborn as sn
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (roc_curve,
                             recall_score,
                             roc_auc_score,
                             accuracy_score,
                             precision_score,
                             f1_score)
from sklearn.ensemble import (AdaBoostClassifier,
                              GradientBoostingClassifier,
                              ExtraTreesClassifier,
                              RandomForestClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')

##### Transformações de categorias para números sequenciais

In [13]:
# Label encoder transforma variáveis categoricas em números sequenciais
le = preprocessing.LabelEncoder() # Instancia objeto chamado LE para o Label Encoder
df_full['Gender'] = le.fit_transform(df_full['Gender'])

In [14]:
df_full['Vehicle_Damage'] = df_full['Vehicle_Damage'].astype('category') # transforma variável em categorica
df_full['Vehicle_Damage'] = df_full['Vehicle_Damage'].cat.codes # gera numeros sequenciais para categorias

##### Get Dummies

 Transforma variável categorica em colunas binárias

In [15]:
df_full = pd.get_dummies(df_full, columns=['Vehicle_Age', 'Driving_License'])

In [16]:
df_full['Meses_segurado'] = round(df_full['Vintage']/30)

#### Divisão do dataset para treinamento

In [17]:
df_full.set_index('id')

,Gender,Age,Region_Code,Previously_Insured,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Driving_License_0,Driving_License_1,Meses_segurado
id,,,,,,,,,,,,,,,
167647,1,22,7.0,1,0,2630.0,152.0,16,0,False,True,False,False,True,1.0
17163,1,42,28.0,0,1,43327.0,26.0,135,0,True,False,False,False,True,4.0
32023,0,66,33.0,0,1,35841.0,124.0,253,0,True,False,False,False,True,8.0
87447,0,22,33.0,0,0,27645.0,152.0,69,0,False,True,False,False,True,2.0
501933,1,28,46.0,1,0,29023.0,152.0,211,0,False,True,False,False,True,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,1,43,39.0,0,1,2630.0,124.0,26,2,True,False,False,False,True,1.0
417524,0,21,12.0,1,0,32937.0,152.0,185,2,False,True,False,False,True,6.0
188087,1,48,29.0,1,0,35247.0,124.0,101,2,True,False,False,False,True,3.0


In [18]:
df_train = df_full.loc[df_full['Response'] != 2] # Tudo que é diferente de 2 volta para o dataset de treinamento
df_test = df_full.loc[df_full['Response'] == 2] # Igual a 2 é o dataset de teste
df_test.drop('Response', axis=1, inplace=True) # Apaga coluna Response do dataset de teste. axis=1(coluna), inplace=True persiste em memoria

In [19]:
X = df_train.drop('Response', axis=1)
y = df_train['Response']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

##### Random Forest com parametros aleatórios

In [21]:
# Random Forest com parametros aleatórios
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

rf = RandomForestClassifier(n_estimators=3,       # Número de árvores do algoritmo (padrão 100)
                            max_depth = 2,        # Padrão None, numeros inteiros. Profundidade máxima da árvore
                            n_jobs=10,            # Número de execuções paralelas
                            min_samples_split=10, # Padrão 2, numeros inteiros numero de divisão de um nó
                            min_samples_leaf=4,   # Padrão 1, numeros inteiros. Numero minimo necessário para estar num nó folha
                            max_features='sqrt')  # “sqrt”, “log2”, None, ou valor inteiro limitado ao número de features

In [22]:
rf.fit(X_train, y_train) # Treinamento do modelo

RandomForestClassifier(max_depth=2, min_samples_leaf=4, min_samples_split=10,
                       n_estimators=3, n_jobs=10)

In [23]:
y_pred = rf.predict(X_test) # predict no dataset de test

In [24]:
def metrics(y_pred, y_test):
    print('Acurácia {}'.format(accuracy_score(y_test, y_pred)))
    print('Recall {}'.format(recall_score(y_test, y_pred)))
    print('F1 Score {}'.format(f1_score(y_test, y_pred)))
    print('Precision {}'.format(precision_score(y_test, y_pred)))

In [25]:
metrics(y_pred, y_test)

Acurácia 0.8344483501530786
Recall 0.0
F1 Score 0.0
Precision 0.0


## Tuning de Modelos

##### Random Search

In [26]:
n_estimator = [1, 50, 100, 150, 200, 250, 300, 400, 500]
min_samples_split = [2, 4, 8, 16]
min_samples_leaf = [1, 2, 4, 8]
max_features = ['auto', 'sqrt', 'log2', None]
max_depth = [2, 4, 6, 8, 10]

# Dicionário com os parametros
rf_params = {
    'n_estimators':n_estimator,
    'min_samples_split': min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'max_features':max_features,
    'max_depth':max_depth
}

In [27]:
rf_tuned = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=37),
                              param_distributions=rf_params, # passagem dos parametros
                              cv=5, # validação cruzada
                              n_iter=5, # numero de interações
                              scoring='precision', # metrica de avaliação
                              n_jobs=-1, # uso de processadores
                              verbose=0) # log

In [28]:
rf_tuned.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=37),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'max_depth': [2, 4, 6, 8, 10],
                                        'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [1, 2, 4, 8],
                                        'min_samples_split': [2, 4, 8, 16],
                                        'n_estimators': [1, 50, 100, 150, 200,
                                                         250, 300, 400, 500]},
                   scoring='precision')

In [29]:
y_pred = rf_tuned.predict(X_test)

In [30]:
metrics(y_pred, y_test)

Acurácia 0.8972149293047355
Recall 0.3791359325605901
F1 Score 0.5498166259168704
Precision 1.0


In [31]:
print('Melhor número de árvores: {}'.format(rf_tuned.best_params_['n_estimators']))
print('Melhor número número mínimo de amostras necessárias para dividir um nó interno: {}'.format(rf_tuned.best_params_['min_samples_split']))
print('Melhor número mínimo de amostras necessárias para estar em um nó da folha: {}'.format(rf_tuned.best_params_['min_samples_leaf']))
print('Melhor número de variáveis a serem considerados ao procurar a melhor divisão: {}'.format(rf_tuned.best_params_['max_features']))
print('Melhor nível de profundidade máxima da arvore: {}'.format(rf_tuned.best_params_['max_depth']))

Melhor número de árvores: 500
Melhor número número mínimo de amostras necessárias para dividir um nó interno: 16
Melhor número mínimo de amostras necessárias para estar em um nó da folha: 1
Melhor número de variáveis a serem considerados ao procurar a melhor divisão: None
Melhor nível de profundidade máxima da arvore: 6


In [34]:
import joblib
joblib.dump(rf_tuned, 'C:/Treinamento-Data-Science/Atividade 4/rf_tuned.pkl')
joblib.dump(rf_tuned, 'C:/Treinamento-Data-Science/Atividade 4/rf_tuned.joblib')

['C:/Treinamento-Data-Science/Atividade 4/rf_tuned.joblib']